In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import pymysql

In [2]:
population_file = "county_facts.csv"
population_df = pd.read_csv(population_file)
population_df.head()

,fips,area_name,state_abbreviation,PST045214,PST040210,PST120214,POP010210,AGE135214,AGE295214,AGE775214,...,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210
0,0,United States,NaN,318857056,308758105,3.3,308745538,6.2,23.1,14.5,...,8.3,28.8,5319456312,4174286516,3917663456,12990,613795732,1046363,3531905.43,87.4
1,1000,Alabama,NaN,4849377,4780127,1.4,4779736,6.1,22.8,15.3,...,1.2,28.1,112858843,52252752,57344851,12364,6426342,13369,50645.33,94.4
2,1001,Autauga County,AL,55395,54571,1.5,54571,6.0,25.2,13.8,...,0.7,31.7,0,0,598175,12003,88157,131,594.44,91.8
3,1003,Baldwin County,AL,200111,182265,9.8,182265,5.6,22.2,18.7,...,1.3,27.3,1410273,0,2966489,17166,436955,1384,1589.78,114.6
4,1005,Barbour County,AL,26887,27457,-2.1,27457,5.7,21.2,16.5,...,0.0,27.0,0,0,188337,6334,0,8,884.88,31.0


In [3]:
pop_df = population_df[['area_name', 'state_abbreviation','POP010210', 'PST045214', 'PST120214']].copy()
pop_df.head()

,area_name,state_abbreviation,POP010210,PST045214,PST120214
0,United States,NaN,308745538,318857056,3.3
1,Alabama,NaN,4779736,4849377,1.4
2,Autauga County,AL,54571,55395,1.5
3,Baldwin County,AL,182265,200111,9.8
4,Barbour County,AL,27457,26887,-2.1


In [4]:
pop_df.columns = ['County', 'State', '2010 Population', '2014 Population', 'Percent Change']
pop_df.head()

,County,State,2010 Population,2014 Population,Percent Change
0,United States,NaN,308745538,318857056,3.3
1,Alabama,NaN,4779736,4849377,1.4
2,Autauga County,AL,54571,55395,1.5
3,Baldwin County,AL,182265,200111,9.8
4,Barbour County,AL,27457,26887,-2.1


In [5]:
pop_df1 = pop_df.dropna(subset=['State'])
pop_df1.head()

,County,State,2010 Population,2014 Population,Percent Change
2,Autauga County,AL,54571,55395,1.5
3,Baldwin County,AL,182265,200111,9.8
4,Barbour County,AL,27457,26887,-2.1
5,Bibb County,AL,22915,22506,-1.8
6,Blount County,AL,57322,57719,0.7


In [6]:
groupby_state2010 = pop_df1.groupby(["State"]).sum()["2010 Population"]
groupby_state2010.head()

State
AK      710231
AL     4779736
AR     2915918
AZ     6392017
CA    37253956
Name: 2010 Population, dtype: int64

In [7]:
groupby_state2014 = pop_df1.groupby(["State"]).sum()["2014 Population"]
groupby_state2014.head()

State
AK      736732
AL     4849377
AR     2966369
AZ     6731484
CA    38802500
Name: 2014 Population, dtype: int64

In [8]:
pop2010 = groupby_state2010.groupby("State").sum().to_frame(name = '2010 Population').reset_index()
pop2010.head()

,State,2010 Population
0,AK,710231
1,AL,4779736
2,AR,2915918
3,AZ,6392017
4,CA,37253956


In [9]:
pop2014 = groupby_state2014.groupby("State").sum().to_frame(name = '2014 Population').reset_index()
pop2014.head()

,State,2014 Population
0,AK,736732
1,AL,4849377
2,AR,2966369
3,AZ,6731484
4,CA,38802500


In [10]:
statepop_df = pop2010.merge(pop2014)
statepop_df.head()

,State,2010 Population,2014 Population
0,AK,710231,736732
1,AL,4779736,4849377
2,AR,2915918,2966369
3,AZ,6392017,6731484
4,CA,37253956,38802500


In [11]:
states_file = "states.csv"
abbrv_df = pd.read_csv(states_file)
abbrv_df.columns = ['State Name', 'State']
abbrv_df.head()

,State Name,State
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [12]:
state_abb_df = abbrv_df.merge(statepop_df)
state_abb_df.head()

,State Name,State,2010 Population,2014 Population
0,Alabama,AL,4779736,4849377
1,Alaska,AK,710231,736732
2,Arizona,AZ,6392017,6731484
3,Arkansas,AR,2915918,2966369
4,California,CA,37253956,38802500


In [13]:
connection_string = "root:MOTigers#21@localhost:3306/pollution_population_db"
engine = create_engine(f'mysql+pymysql://{connection_string}')

In [14]:
state_abb_df.to_sql(name='state_populations', con=engine, if_exists='append', index=False)